In [ ]:
!pip install -qqq git+https://github.com/wandb/weave.git@verdict-integration
!pip install -qqq -U git+https://github.com/parambharat/verdict.git@feat/tracing-impl

In [ ]:
import wandb
wandb.login()

In [ ]:
from verdict import Pipeline
from verdict.schema import Schema
from verdict.common.judge import JudgeUnit
import weave
from getpass import getpass
import os

In [ ]:
os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API Key: ")

In [ ]:
weave.init("verdict-example")

In [ ]:
# ref: https://github.com/haizelabs/verdict/blob/main/notebooks/common/judge.ipynb
# default scale is DiscreteScale((1, 5))
pipeline = Pipeline() \
    >> JudgeUnit().prompt("""
        Score this on how funny it is.

        {source.joke}
    """)

response, leaf_node_prefixes = pipeline.run(Schema.of(joke="Why did the chicken cross the road? To get to the other side."))
print(response)

In [ ]:
from verdict import Pipeline, Layer
from verdict.common.conversational import ConversationalUnit
from verdict.schema import Schema

pipeline = Pipeline() \
>> Layer(
    ConversationalUnit("Thinker").prompt("""
        Is this joke funny? Read arguments from those before you and come up with a brief, but unique, stance.

        Joke:
        {source.joke}

        Others' thoughts:
        {input.conversation}
    """)
, 4, inner='chain')

response, _ = pipeline.run(Schema.of(joke="Why did the chicken cross the road? Because the other side had better documentation."))